In [3]:
import pandas as pd
import numpy as np

In [18]:
data=pd.read_csv('../2~3.데이터 수집 및 전처리/머신러닝 전처리 데이터.CSV',encoding='EUC-KR')

In [19]:
data

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.3100,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.9800,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000
2,광주광역시 광산구 도산동,대주피오레1차,84.9800,202305,31,2023-05-31,5,23,16,3.5,2920010200,1,22350
3,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,9,2023-05-09,5,22,16,3.5,2920010200,1,22400
4,광주광역시 광산구 도산동,대주피오레2차,84.7575,202305,13,2023-05-13,13,22,16,3.5,2920010200,1,22000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425074,광주광역시 서구 화정동,화정우미,115.1850,200611,10,2006-11-10,17,10,131,4.5,2914011900,1,19000
425075,광주광역시 서구 화정동,화정우미,84.8300,200611,27,2006-11-27,6,10,131,4.5,2914011900,1,12000
425076,광주광역시 서구 화정동,화정우미,59.8300,200611,30,2006-11-30,2,10,131,4.5,2914011900,1,7700
425077,광주광역시 서구 화정동,화정우미,84.8400,200612,6,2006-12-06,16,10,124,4.5,2914011900,1,12170


In [8]:
pip install xgboost

                                              0.0/70.9 MB ? eta -:--:--
                                              0.1/70.9 MB 2.6 MB/s eta 0:00:27
                                              0.3/70.9 MB 4.2 MB/s eta 0:00:17
                                              0.6/70.9 MB 4.2 MB/s eta 0:00:17
                                              0.8/70.9 MB 4.5 MB/s eta 0:00:16
                                              0.9/70.9 MB 4.4 MB/s eta 0:00:16
                                              1.2/70.9 MB 4.2 MB/s eta 0:00:17
                                              1.5/70.9 MB 4.5 MB/s eta 0:00:16
                                              1.8/70.9 MB 4.5 MB/s eta 0:00:16
     -                                        2.0/70.9 MB 4.5 MB/s eta 0:00:16
     -                                        2.3/70.9 MB 4.8 MB/s eta 0:00:15
     -                                        2.4/70.9 MB 4.7 MB/s eta 0:00:15
     -                                        2.7/70.9 MB 4

In [10]:
pip install lightgbm

                                              0.0/1.3 MB ? eta -:--:--
     ----------                               0.3/1.3 MB 7.0 MB/s eta 0:00:01
     -----------------------------            1.0/1.3 MB 8.6 MB/s eta 0:00:01
     ---------------------------------------  1.3/1.3 MB 9.2 MB/s eta 0:00:01
     ---------------------------------------- 1.3/1.3 MB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install tqdm

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import re

In [13]:
data.head(2)

,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
0,광주광역시 광산구 도산동,대덕1,59.31,202303,18,2023-03-18,4,33,20,3.5,2920010200,1,9800
1,광주광역시 광산구 도산동,대주피오레1차,84.98,202302,7,2023-02-07,2,23,14,3.5,2920010200,1,21000


In [8]:
data[data['시군구']=='광주광역시 남구 봉선동']

,Unnamed: 0,시군구,단지명,전용면적(㎡),계약년월,계약일,계약날짜,층,건축나이,거래건수,기준금리,법정동코드,지하철역수,거래금액(만원)
2104,2104,광주광역시 남구 봉선동,금호2,122.8100,202301,12,2023-01-12,15,28,29,3.5,2915511500,0,64000
2105,2105,광주광역시 남구 봉선동,금호2,122.8100,202301,15,2023-01-15,1,28,29,3.5,2915511500,0,60500
2106,2106,광주광역시 남구 봉선동,금호2,84.9600,202301,21,2023-01-21,6,28,29,3.5,2915511500,0,49500
2107,2107,광주광역시 남구 봉선동,금호2,84.8300,202303,25,2023-03-25,6,28,27,3.5,2915511500,0,49500
2108,2108,광주광역시 남구 봉선동,금호2,84.8300,202304,18,2023-04-18,13,28,27,3.5,2915511500,0,49000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406940,406940,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,23,2006-12-23,5,2,100,4.5,2915511500,0,13650
406941,406941,광주광역시 남구 봉선동,포스코더샵,59.8900,200612,27,2006-12-27,17,2,100,4.5,2915511500,0,13000
406942,406942,광주광역시 남구 봉선동,포스코더샵,84.3200,200612,27,2006-12-27,8,2,100,4.5,2915511500,0,23900
406943,406943,광주광역시 남구 봉선동,포스코더샵,150.9700,200612,27,2006-12-27,2,2,100,4.5,2915511500,0,41000


In [6]:
data['시군구']

0         광주광역시 광산구 도산동
1         광주광역시 광산구 도산동
2         광주광역시 광산구 도산동
3         광주광역시 광산구 도산동
4         광주광역시 광산구 도산동
              ...      
425074     광주광역시 서구 화정동
425075     광주광역시 서구 화정동
425076     광주광역시 서구 화정동
425077     광주광역시 서구 화정동
425078     광주광역시 서구 화정동
Name: 시군구, Length: 425079, dtype: object

In [20]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]
y = data['거래금액(만원)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [131]:
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)

In [132]:
def print_rmse_r2_score(model):
    # cv별 프린팅, 평균 저장
    model_name, rmse_list, r2_list = rmse_r2_cv(model)
    for i, (rmse, r2) in enumerate(zip(rmse_list, r2_list), start=1):
         print(f'{i} FOLDS: {model_name} RMSE: {rmse:.4f}, R2: {r2:.4f}')
    print(f'\n{model_name} mean RMSE: {np.mean(rmse_list):.4f}, mean R2: {np.mean(r2_list):.4f}')
    print('='*40)
    return model_name, np.mean(rmse_list), np.mean(r2_list)

def rmse_r2_cv(model):
    # cv별로 학습하는 함수
    tscv = TimeSeriesSplit(n_splits=10)
    rmse_list = []
    r2_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(X), start=1), desc=f'{model_name} Cross Validations...', total=10):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        rmse = RMSE(y_test, pred)
        r2 = r2_score(y_test, pred)
        rmse_list.append(rmse)
        r2_list.append(r2)
    return model_name, rmse_list, r2_list



In [ ]:
models = []
rmse_scores = []
r2_scores = []
for model in [lrmodel, ridge, lasso, enet, d_tree, rf, model_xgb, model_lgb]:
    model_name, mean_rmse_score, mean_r2_score = print_rmse_r2_score(model)
    models.append(model_name)
    rmse_scores.append(mean_rmse_score)
    r2_scores.append(mean_r2_score)

LinearRegression Cross Validations...: 100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.22it/s]


1 FOLDS: LinearRegression RMSE: 10084.0162, R2: 0.6496
2 FOLDS: LinearRegression RMSE: 9080.6999, R2: 0.5288
3 FOLDS: LinearRegression RMSE: 6591.1190, R2: 0.6617
4 FOLDS: LinearRegression RMSE: 5906.4291, R2: 0.5981
5 FOLDS: LinearRegression RMSE: 5854.3037, R2: 0.4538
6 FOLDS: LinearRegression RMSE: 5383.9206, R2: 0.5477
7 FOLDS: LinearRegression RMSE: 5841.3210, R2: 0.5347
8 FOLDS: LinearRegression RMSE: 8988.2098, R2: -0.3882
9 FOLDS: LinearRegression RMSE: 4756.1851, R2: 0.2545
10 FOLDS: LinearRegression RMSE: 4739.9987, R2: 0.0180

LinearRegression mean RMSE: 6722.6203, mean R2: 0.3859


Ridge Cross Validations...: 100%|██████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.93it/s]


1 FOLDS: Ridge RMSE: 10084.0369, R2: 0.6496
2 FOLDS: Ridge RMSE: 9080.6665, R2: 0.5288
3 FOLDS: Ridge RMSE: 6591.1186, R2: 0.6617
4 FOLDS: Ridge RMSE: 5906.4287, R2: 0.5981
5 FOLDS: Ridge RMSE: 5854.3037, R2: 0.4538
6 FOLDS: Ridge RMSE: 5383.9201, R2: 0.5478
7 FOLDS: Ridge RMSE: 5841.3208, R2: 0.5347
8 FOLDS: Ridge RMSE: 8988.2091, R2: -0.3882
9 FOLDS: Ridge RMSE: 4756.1838, R2: 0.2545
10 FOLDS: Ridge RMSE: 4739.9990, R2: 0.0180

Ridge mean RMSE: 6722.6187, mean R2: 0.3859


Lasso Cross Validations...: 100%|██████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.72it/s]


...

ElasticNet Cross Validations...: 100%|█████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.60it/s]


1 FOLDS: ElasticNet RMSE: 10187.8392, R2: 0.6423
2 FOLDS: ElasticNet RMSE: 8660.3586, R2: 0.5714
3 FOLDS: ElasticNet RMSE: 6574.6227, R2: 0.6634
4 FOLDS: ElasticNet RMSE: 5887.6433, R2: 0.6007
5 FOLDS: ElasticNet RMSE: 5846.1615, R2: 0.4553
6 FOLDS: ElasticNet RMSE: 5350.0822, R2: 0.5534
7 FOLDS: ElasticNet RMSE: 5822.6967, R2: 0.5377
8 FOLDS: ElasticNet RMSE: 8951.1368, R2: -0.3768
9 FOLDS: ElasticNet RMSE: 4650.9873, R2: 0.2871
10 FOLDS: ElasticNet RMSE: 4763.4061, R2: 0.0083

ElasticNet mean RMSE: 6669.4935, mean R2: 0.3943


DecisionTreeRegressor Cross Validations...: 100%|██████████████████████████████████████| 10/10 [00:23<00:00,  2.36s/it]


1 FOLDS: DecisionTreeRegressor RMSE: 12018.0958, R2: 0.5023
2 FOLDS: DecisionTreeRegressor RMSE: 7239.8274, R2: 0.7005
3 FOLDS: DecisionTreeRegressor RMSE: 5952.2796, R2: 0.7241
4 FOLDS: DecisionTreeRegressor RMSE: 4816.5577, R2: 0.7327
5 FOLDS: DecisionTreeRegressor RMSE: 3828.5220, R2: 0.7664
6 FOLDS: DecisionTreeRegressor RMSE: 3381.4206, R2: 0.8216
7 FOLDS: DecisionTreeRegressor RMSE: 4799.1985, R2: 0.6859
8 FOLDS: DecisionTreeRegressor RMSE: 3768.8259, R2: 0.7559
9 FOLDS: DecisionTreeRegressor RMSE: 2756.5221, R2: 0.7496
10 FOLDS: DecisionTreeRegressor RMSE: 2483.8391, R2: 0.7303

DecisionTreeRegressor mean RMSE: 5104.5089, mean R2: 0.7169


RandomForestRegressor Cross Validations...:  90%|█████████████████████████████▋   | 9/10 [8:59:28<1:56:11, 6971.90s/it]

In [ ]:
result_df = pd.DataFrame({'Model': models, 'RMSE': rmse_scores, 'R2': r2_scores}).reset_index(drop=True)
result_df

In [ ]:
주광역시 광산구 도산동	대주피오레1차	84.9800	202305	31	2023-05-31	5	23	16	3.5	2920010200	1	22350

In [129]:
X_predict = [[84, 202305,10,28 ,15,3.5,2920010200,0]]
X_predict_df= pd.DataFrame(X_predict,columns=['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수'])

In [121]:
X = data[['전용면적(㎡)', '계약년월','층', '건축나이','거래건수','기준금리','법정동코드','지하철역수']]

In [130]:
print(f"LinearRegression: {lrmodel.predict(X_predict)[0]}")
print(f"Ridge: {ridge.predict(X_predict)[0]}")
print(f"Lasso: {lasso.predict(X_predict)[0]}")
print(f"ElasticNet: {enet.predict(X_predict)[0]}")
print(f"DecisionTreeRegressor: {d_tree.predict(X_predict)[0]}")
print(f"RandomForestRegressor: {rf.predict(X_predict)[0]}")
print(f"XGBRegressor: {model_xgb.predict(X_predictXGB)[0]}")
print(f"LGBMRegressor :{model_lgb.predict(X_predict)[0]}")

LinearRegression: 26323.957539539784
Ridge: 26323.956001826562
Lasso: 26323.916086268146
ElasticNet: 26302.825872001704
DecisionTreeRegressor: 20556.70123900879
RandomForestRegressor: 18380.31887551234
XGBRegressor: 29209.294921875
LGBMRegressor :16672.703923155033


C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(
C:\Users\gjaischool1\anaconda3\envs\machine\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
C:\Users\gjaischoo

In [ ]:
lrmodel = LinearRegression()
ridge = Ridge()
lasso = Lasso()
enet = ElasticNet()
d_tree = DecisionTreeRegressor()
rf = RandomForestRegressor()
model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()

In [ ]:
lrmodel = LinearRegression(n_jobs=-1)
ridge = Ridge(alpha=0.8, random_state=1)
lasso = Lasso(alpha = 0.01, random_state=1)
enet = ElasticNet(alpha=0.03, l1_ratio=0.01, random_state=1)
d_tree = DecisionTreeRegressor(max_depth=6, min_samples_split=10, min_samples_leaf=15, random_state=1)
rf = RandomForestRegressor(n_estimators=100,  max_depth=9, min_samples_split=50,
                           min_samples_leaf=5, random_state=1, n_jobs=-1)
model_xgb = xgb.XGBRegressor(n_estimators=100, max_depth=9, min_child_weight=5, gamma=0.1, n_jobs=-1 )
model_lgb = lgb.LGBMRegressor(n_estimators=100, max_depth=9, min_child_weight=5, n_jobs=-1,num_leaves=20)